# Logistic Regression Model

## 1. Loading all the libraries needed

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.utils import AnalysisException

## 2. Setting up Spark and loading the data

In [2]:
# Initializing Spark

spark = SparkSession.builder \
    .master("local[8]") \
    .appName("LogisticRegression") \
    .config("spark.executor.memory", "1gb") \
    .getOrCreate()

# Loading the data
try:
    file = spark.read.format("csv").option("header", "true").option("multiline", "true").option("quote", "\"").option("escape", "\"").load("../Final Preprocessing/final.csv")
except Exception as e:
    print("Error occured during loading the file:", e)

## 3. Feature selection and transformation(with numerical variables as features for the regression model)

As the first step in our analysis, we use the numerical columns in our dataset as inputs in the logistic regression model. Now, we need to make sure the input variables in the logistic regression model is ready to work with. Logistic regression models needs input variables to be numerical. However, in our dataset, these columns that should contain numerical information are read as strings. So, we transform the type of data by using **withColumn** in PySpark as it allows us to modify the datatypes in our target columns nicely. We use it to cast each of the selected columns into "double" datatype, making sure that all the features are numerical and suitable for the logistic regression. As logistic regression also needs binary numerical variable as the output variable, we cast the "billboard" into a double.

### 3.1. Selecting the features and transforming data types

In [3]:
# Including numeric columns as features in the regression model(they contain numeric information but their datatype right now is strings)
numeric_columns = ["popularity",
                   "duration_ms", 
                   "danceability", 
                   "energy", 
                   "key", 
                   "loudness",
                   "mode", 
                   "speechiness", 
                   "acousticness", 
                   "instrumentalness", 
                   "liveness", 
                   "valence",
                   "tempo", 
                   "time_signature"]

try:
    # Converting the string columns to numeric
    for column in numeric_columns:
        file = file.withColumn(column, col(column).cast("double"))
        
except AnalysisException as e:
    print("Error when transforming the numeric columns:", e)
    
    
try:
    # Converting the billboard column to numeric
     file = file.withColumn("billboard", col("billboard").cast("double"))

except AnalysisException as e:
    print("Error when transforming the billboard column:", e)

### 3.2. Dropping potential NA's

To ensure reliability and integrity in our model, we check for and drop any null values contained in the numeric_columns.

In [4]:
# Checking for null values in the numeric columns and drop them
file = file.dropna(subset=numeric_columns)

### 3.3. Assembling the features

In logistic regression, the model expects a single vector that contains all the features. That is why, in the next step, we use **VectorAssembler** to combine all the feature columns(numeric_columns) into a single vector. We name this single vector as "unscaledFeatures" as this will make it easier to distinguish in our analysis. 

In [5]:
try: 
    # Assembling all the features into one vector
    assembler = VectorAssembler(inputCols = numeric_columns, outputCol = "unscaledFeatures")
    file = assembler.transform(file)
    
except Exception as e:
    print("Error when assembling the features:", e)

## 4. Scaling the features


Next, we scale the features we have chosen to further prepare it for to be used in the model. Scaling is an important step as it will help make sure that all features contribute equally to the model by normalizing the data, which then will lead to better performance of the model overall. We scale the features here using **StandardScaler** in PySpark.

In [6]:
try:
    # Scaling the features
    scaler = StandardScaler(inputCol = "unscaledFeatures", outputCol = "scaledFeatures")
    file = scaler.fit(file).transform(file)
    
except Exception as e:
    print("Error when scaling the features:", e)

## 5. Splitting into training and test dataset (Starting with the entire data)


We now split our dataset into training and test data with 80/20 split. We defined the speficic seed for reproducibility and comparability with our other models. 

In [7]:
# Splitting data into training and testing sets (using the entire data)
train_data, test_data = file.randomSplit([0.8, 0.2], seed = 1234)

## 5.1. Alternative(Creating balanced dataset)


As our dataset includes many more songs that didn't end up on billboard than the ones that did, there will be some bias of any models built on that dataset to be classified as "non-billboard". Thus, to potentially avoid this issue and to take the bias into account, we also create a balanced dataset with 1:1 distribution of both classes. We name this dataframe, "balanced_df", and from this step onwards, we will calculate everything on both the original data and this balanced data to get the whole picture. The downside of creating this evenly split dataset may be that a lot of entries in our data with songs that are not on billboard gets eliminated. 

In [8]:
# credit to Moritz

label_counts = file.groupBy("billboard").count().collect()

min_count = min(row["count"] for row in label_counts)

# Create balanced DataFrame by sampling
balanced_df = None

for row in label_counts:
    label = row["billboard"]
    count = row["count"]
    fraction = min_count / count  # Calculate the fraction of the data to sample
    
    # Sample the data
    sampled_df = file.filter(col("billboard") == label).sample(False, fraction, seed=1)
    
    # Append sampled data to the balanced DataFrame
    if balanced_df is None:
        balanced_df = sampled_df.limit(min_count)  # Use limit to ensure exact number of instances
    else:
        balanced_df = balanced_df.union(sampled_df.limit(min_count))

In [9]:
balanced_df.select("billboard").groupBy("billboard").count().show()

+---------+-----+
|billboard|count|
+---------+-----+
|      0.0| 8825|
|      1.0| 8923|
+---------+-----+



## 5.2. Splitting into training and test data (Starting with the balanced data)

After creating the balanced dataset, we split that data again into training and test data with 80/20 split.

In [10]:
# Splitting data into training and testing sets (using the balanced data)
train_balanced_data, test_balanced_data = balanced_df.randomSplit([0.8, 0.2], seed = 1234)

## 6. Model Training 


We now define our logistic regression model. We use the following parameters for it: 
- **scaledFeatures** as input features. As we know, they have been preprocessed and scaled.
- **billboard** as output label. This column contains the target variable of whether a song is on billboard(1) or not(0).

In [11]:
# Defining logistic regression model
lr = LogisticRegression(featuresCol = "scaledFeatures", labelCol = "billboard")

### 6.1. Defining Parameters and Cross Validation


In this step, we definde a range of parameters for the logistic regression model. By doing so, we can achieve the optimal model performance. We use **ParamGridBuilder** to build a grid of parameter combinations that will be tested during the cross-validation. This helps us test the logistic regression model accross different settings and find the best set of parameters for it. 

In [12]:
# Creating the parameter grid with a large range of parameters to explore different levels of regularization
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .build()

# https://spark.apache.org/docs/latest/ml-tuning.html

Next we use **CrossValidator** for tuning parameters and selecting the models. This means we split the dataset into a number of folds and use the subset of the folds to train the model, and validating on the remaining fold. We've chosen 3-fold cross validation. This step is important to finding the logistic regression model with the best performance. 

In [13]:
try:
    # Using CrossValidator to find the best model
    crossval = CrossValidator(estimator = lr,
                              estimatorParamMaps = paramGrid,
                              evaluator = BinaryClassificationEvaluator(labelCol = "billboard"),
                              numFolds = 3)
    # Running cross-validation and choosing the best set of parameters
    cvModel = crossval.fit(train_data)  # using entire data
    cvModel_balanced = crossval.fit(train_balanced_data)  # using the balanced data

except Exception as e:
    print("Error when training the model:", e)


### 6.2. Predictions

Then, after running the cross-validations, we fetch the best modesl resulted from it. Next, we make predictions using the best models. We do this for both the test data from the entire dataset and the test data from the balanced dataset to have a wider outlook.

In [14]:
# Getting the best models resulting from the cross-validation step
bestModel = cvModel.bestModel
bestModel_balanced = cvModel_balanced.bestModel

# Making predictions on the test data
predictions = bestModel.transform(test_data)  # using entire data
predictions_balanced = bestModel_balanced.transform(test_balanced_data)  # using balanced data

## 7. Model Evaluation

We use **BinaryClassificationEvaluator** in our logistic regression model to evaluate our models' performance. This evaluator is suitable to use in the logistic regression since our target variable is a binary variable with values 0 and 1. The configurations used are:

- **labelCol** contains the true labels for the classification. In our case, it's the "billboard_numeric" column.
- **rawPredictionCol** specifies the column containing raw prediction scores from the model
- **metricName** specifies the metric used for evaluation of the model. The **areaUnderROC** evaluates model's ability to distinguish between positive and negative classes, with 1 indicating perfect classification. 

In [15]:
# Defining the evaluator for binary classification - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html
evaluator = BinaryClassificationEvaluator(labelCol = "billboard", rawPredictionCol = "rawPrediction", metricName = "areaUnderROC")

### 7.1. Calculating areaUnderROC and areaUnderPR

In [16]:
# Evaluating the model - using the entire data
areaUnderROC = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
areaUnderPR = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

print("Test set AUC (ROC) for entire data = " + str(areaUnderROC))
print("Test set AUC (PR) for entire data = " + str(areaUnderPR))


# Evaluate the model - using the entire data
areaUnderROC_balanced = evaluator.evaluate(predictions_balanced, {evaluator.metricName: "areaUnderROC"})
areaUnderPR_balanced = evaluator.evaluate(predictions_balanced, {evaluator.metricName: "areaUnderPR"})

print("Test set AUC (ROC) for balanced data = " + str(areaUnderROC))
print("Test set AUC (PR) for balanced data = " + str(areaUnderPR))

Test set AUC (ROC) for entire data = 0.6915915065465859
Test set AUC (PR) for entire data = 0.26414429593305294
Test set AUC (ROC) for balanced data = 0.6915915065465859
Test set AUC (PR) for balanced data = 0.26414429593305294


### Accuracy

Since the BinaryClassificationEvaluator doesn't support **Accuracy** as an evaluator metric, we need to manually calculate the accuracy of our model here. **Accuracy** is calculated by dividing the number of correct predictions by the total number of predictions. What we notice here is that the accuracy for the balanced dataset is much lower than the accuracy calculated by using the entire dataset. This is completely expected result, since our original dataset contains much more 0's in target variable than 1's, and when trained using the skewed data, it will predict the large portion to be 0, which will turn out accurate given the prevalence of 0s in the original dataset. 

In [18]:
# Calculating accuracy 
accuracy = predictions.filter(predictions.billboard == predictions.prediction).count() / float(predictions.count())
print(f"Accuracy (Entire Dataset): {accuracy}")

accuracy_balanced = predictions_balanced.filter(predictions_balanced.billboard == predictions_balanced.prediction).count() / float(predictions_balanced.count())
print(f"Accuracy (Balanced Dataset): {accuracy_balanced}")

Accuracy (Entire Dataset): 0.8604538087520259
Accuracy (Balanced Dataset): 0.6246466930469191


### 7.2. Confusion Matrix

For further inspection and visualization of the model performance, we create **Confusion Matrix** for both the entire dataset and balanced dataset results. Some notable insights include:
- The balanced training seems to have largely improved the ability of the model to identify songs on the billboard, by reducing the False Negatives(FN) and increasing the True Positives(TP)
- However, there is also a large increase in False Positives(FP) and this hints us the trade-off between making the model less conservative and making more positive predictions but at the cost of increased mistake in that area of prediction. 
- Overall, the model seem to have much more balanced predictions after eliminating the skewness of the original data. 

In [19]:
# Confusion Matrix
confusion_matrix = predictions.groupBy("billboard", "prediction").count()
confusion_matrix.show()

confusion_matrix_balanced = predictions_balanced.groupBy("billboard", "prediction").count()
confusion_matrix_balanced.show()

+---------+----------+-----+
|billboard|prediction|count|
+---------+----------+-----+
|      1.0|       1.0|    8|
|      0.0|       1.0|   12|
|      1.0|       0.0| 1710|
|      0.0|       0.0|10610|
+---------+----------+-----+

+---------+----------+-----+
|billboard|prediction|count|
+---------+----------+-----+
|      1.0|       1.0| 1158|
|      0.0|       1.0|  688|
|      1.0|       0.0|  640|
|      0.0|       0.0| 1052|
+---------+----------+-----+



### 7.3. Precision, Recall, and F1 Scores ( Additional Evaluation Metrics)

On top of ROC-AUC, PR-AUC, and accuracy, making use of the confusion matrix we created, we can also calculate additional metrics for evaluation such as precision, recall, and F1 scores based on our balanced dataset, so that we have more broader picture when benchmarking the results with our other models. 

In [20]:
# Assigning the needed values from the confusion matrix

TP = confusion_matrix_balanced.filter((col("billboard") == 1) & (col("prediction") == 1)).collect()[0]["count"]
FP = confusion_matrix_balanced.filter((col("billboard") == 0) & (col("prediction") == 1)).collect()[0]["count"]
FN = confusion_matrix_balanced.filter((col("billboard") == 1) & (col("prediction") == 0)).collect()[0]["count"]

# Calculating Precision and Recall

Precision_balanced = TP/(TP + FP) 
Recall_balanced = TP/(TP + FN) 
F1_balanced = 2 * (Precision_balanced * Recall_balanced) / (Precision_balanced + Recall_balanced)


print(f"Precision(Balanced Dataset): {Precision_balanced}")
print(f"Recall(Balanced Dataset): {Recall_balanced}")
print(f"F1 Score(Balanced Dataset): {F1_balanced}")

Precision(Balanced Dataset): 0.6273022751895991
Recall(Balanced Dataset): 0.6440489432703004
F1 Score(Balanced Dataset): 0.6355653128430296


## 8. Getting the coefficients of the features based on the best model

What can also offer us extra insights into the model and its predictions is finding out the coefficients of each features within the model. By sorting in order of magnitude of the coefficients, we can get a insight on which variables have more influence on whether a song ends up on billboard. In other words, which characteristics of a song are more important to decide what ends up on billboard. On top of providing us with the magnitude of a feature's influence on the output variable, it can also show us whether that influence is negative or positive. 

In [21]:
# Extracting coefficients and intercepts
coefficients = bestModel_balanced.coefficients # used only the balanced dataset here as both of them(entire and balanced) showed very comparable results
intercept = bestModel_balanced.intercept  # intercept tells us log-odds of target variable when all predictors are 0.

In [22]:
# Mapping coefficients to feature names
featureNames = numeric_columns
featureCoefficients = [(feature, coeff) for feature, coeff in zip(featureNames, coefficients)]
sorted_features = sorted(featureCoefficients, key = lambda x: abs(x[1]), reverse = True)  # sorting from the most to least important

# Printing sorted features by importance
print("Features, sorted by their importance:")
for feature, coefficient in sorted_features:
    print(f"{feature}: {coefficient}")

Features, sorted by their importance:
valence: 0.42657706555151625
instrumentalness: -0.4224917545190806
loudness: -0.3808035945347383
acousticness: -0.37450232777374254
energy: -0.17638056347508896
mode: 0.1598584359155145
liveness: -0.15633697691839823
popularity: 0.08315368262359175
tempo: -0.07379338312891608
danceability: -0.049528215892181704
speechiness: -0.04882585617490412
duration_ms: -0.03614428911309265
time_signature: -0.007931457372988714
key: 0.0027720485185212407


From the sorted features based on their coefficients, we see that **valence, instumentalness, acousticness, loudness, energy, mode, liveliness** seems to have more influence on the probability of a song ending up on billboard than the other feature variables. 

## 9. Saving the model

In [23]:
from py4j.protocol import Py4JJavaError

try:
    bestModel_balanced.save("../Final Code/trained_models/LR_model")
    
except Py4JJavaError:
    print("Error saving model the model, make sure the model isn't allready saved in the defined folder")